In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import cv2
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import os
from skimage.feature import local_binary_pattern, hog
from sklearn.ensemble import RandomForestClassifier

## Load & Preprocessing Image

In [ ]:
DATASET_PATH = '/content/drive/My Drive/dataset-coral-baru'

In [ ]:
IMAGE_SIZE = (224, 224)
RADIUS = 1
N_POINTS = 8 * RADIUS

images = []
class_list = []

clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))

for class_index, class_name in enumerate(os.listdir(DATASET_PATH)):
    class_path = os.path.join(DATASET_PATH, class_name)
    if os.path.isdir(class_path):
        for file_name in os.listdir(class_path):
            file_path = os.path.join(class_path, file_name)

            image = cv2.imread(file_path)
            if image is not None:

                image = cv2.resize(image, IMAGE_SIZE)

                gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

                gray_image = clahe.apply(gray_image)


                images.append(gray_image)
                class_list.append(class_index)

images = np.array(images)
class_list = np.array(class_list)

print(f"Total images loaded: {len(images)}")
print(f"Total classes: {len(np.unique(class_list))}")

Total images loaded: 1218
Total classes: 11


## Feature Extraction

In [ ]:
#Library
from skimage.feature import hog, graycomatrix, graycoprops
from skimage import img_as_ubyte

### HOG

In [ ]:
#Parameter
ORIENTATIONS8 = 8
ORIENTATIONS9 = 9
PIXEL_PER_CELL16 = (16,16)
PIXEL_PER_CELL8 = (8,8)

In [ ]:
def HOG(data):
  HOG_FEATURES = []
  for image in data:
    features = hog(image,
                   orientations=8,
                   pixels_per_cell=(16,16),
                   cells_per_block=(2,2),
                   block_norm='L2-Hys',
                   visualize= False)
    HOG_FEATURES.append(features)
  return HOG_FEATURES



### Color Histogram

In [ ]:
def Color_HIST(data):
    HIST_FEATURES = []
    for image in data:

        image_rgb = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)

        image_hsv = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2HSV)

        hist_h = cv2.calcHist([image_hsv], [0], None, [256], [0, 256])
        hist_s = cv2.calcHist([image_hsv], [1], None, [256], [0, 256])
        hist_v = cv2.calcHist([image_hsv], [2], None, [256], [0, 256])

        hist_h /= hist_h.sum()
        hist_s /= hist_s.sum()
        hist_v /= hist_v.sum()

        features = np.concatenate((hist_h, hist_s, hist_v), axis=0)

        HIST_FEATURES.append(features.flatten())

    return np.array(HIST_FEATURES)


### LBP

In [ ]:
RADIUS1 = 1
RADIUS2 = 2

N_POINTS8 = 8
N_POINTS16 = 16
N_POINTS24 = 24


In [ ]:
def lbp_histogram(lbp_image, num_bins=59):
    lbp_hist, _ = np.histogram(lbp_image.ravel(), bins=np.arange(0, num_bins + 1), range=(0, num_bins))
    lbp_hist = lbp_hist.astype('float')
    lbp_hist /= lbp_hist.sum()
    return lbp_hist

lbp_features = []
for image in images:
    lbp = local_binary_pattern(image, N_POINTS, RADIUS, method='uniform')
    lbp_hist = lbp_histogram(lbp)
    lbp_features.append(lbp_hist)




### GLCM

In [ ]:
def GLCM(data, distances=[1], angles=[0, np.pi/4, np.pi/2, 3*np.pi/4], levels=256):
    GLCM_FEATURES = []

    for image in data:

        image_uint8 = img_as_ubyte(image)

        glcm = graycomatrix(image_uint8, distances=distances, angles=angles, levels=levels, symmetric=True, normed=True)

        contrast = graycoprops(glcm, prop='contrast').mean()
        correlation = graycoprops(glcm, prop='correlation').mean()
        energy = graycoprops(glcm, prop='energy').mean()
        homogeneity = graycoprops(glcm, prop='homogeneity').mean()

        features = [contrast, correlation, energy, homogeneity]

        GLCM_FEATURES.append(features)

    return np.array(GLCM_FEATURES)


### Gabor Filters

In [ ]:
def Gabor_Filters(data, ksize=31, sigma=4.0, lambda_=10.0, gamma=0.5, phi=0, orientations=[0, np.pi/4, np.pi/2, 3*np.pi/4]):
    GABOR_FEATURES = []

    for image in data:

        features_per_image = []

        for theta in orientations:

            gabor_kernel = cv2.getGaborKernel((ksize, ksize), sigma, theta, lambda_, gamma, phi)

            filtered_image = cv2.filter2D(image, cv2.CV_8UC3, gabor_kernel)

            energy = np.sum(filtered_image**2)
            mean = np.mean(filtered_image)
            variance = np.var(filtered_image)

            features_per_image.extend([energy, mean, variance])

        GABOR_FEATURES.append(features_per_image)

    return np.array(GABOR_FEATURES)


In [ ]:
hog_features = HOG(images)
hog_features = np.array(hog_features)
glcm_features = GLCM(images)
gabor_features = Gabor_Filters(images)

lbp_features = np.array(lbp_features)
color_hist_features = Color_HIST(images)


In [ ]:
print(f"LBP Features Shape: {lbp_features.shape}")
print(f"HOG Features Shape: {hog_features.shape}")
print(f"GLCM Features Shape: {glcm_features.shape}")
print(f"Gabor Features Shape: {gabor_features.shape}")
print(f"Color Hist Features Shape: {color_hist_features.shape}")

LBP Features Shape: (1218, 59)
HOG Features Shape: (1218, 5408)
GLCM Features Shape: (1218, 4)
Gabor Features Shape: (1218, 12)
Color Hist Features Shape: (1218, 768)


### Combined_features1 = HOG + LBP + GLCM
### Combined_features2 = Color Histogram + Gabor Filters + LBP
### Combined_features3 = HOG + LBP + SIFT + Color Histogram

In [ ]:
combined_features = np.concatenate((gabor_features,hog_features,color_hist_features),axis=1)
combined_features2 = np.concatenate((lbp_features,hog_features,glcm_features),axis=1)
combined_features3 = np.concatenate((lbp_features,gabor_features,color_hist_features),axis=1)
combined_features4 = np.concatenate((lbp_features,hog_features,color_hist_features),axis=1)

combined_features5 = np.concatenate((lbp_features,color_hist_features),axis=1)
combined_features6 = np.concatenate((gabor_features,hog_features),axis=1)
combined_features7 = np.concatenate((gabor_features,color_hist_features),axis=1)
combined_features8 = np.concatenate((lbp_features,color_hist_features),axis=1)




In [ ]:
x_train_hog, x_test_hog, y_train_hog, y_test_hog = train_test_split(hog_features, class_list, test_size=0.2, random_state=42)
x_train_lbp, x_test_lbp, y_train_lbp, y_test_lbp = train_test_split(lbp_features, class_list, test_size=0.2, random_state=42)
x_train_glcm, x_test_glcm, y_train_glcm, y_test_glcm = train_test_split(glcm_features, class_list, test_size=0.2, random_state=42)
x_train_gabor, x_test_gabor, y_train_gabor, y_test_gabor = train_test_split(gabor_features, class_list, test_size=0.2, random_state=42)
x_train_color_hist, x_test_color_hist, y_train_color_hist, y_test_color_hist = train_test_split(color_hist_features, class_list, test_size=0.2, random_state=42)




In [ ]:
X_train, X_test, y_train, y_test = train_test_split(combined_features, class_list, test_size=0.2, random_state=42)
print(f"Shape of X_train (Combined Features Gabor + HOG + Color Hist): {X_train.shape}")

X_train2, X_test2, y_train2, y_test2 = train_test_split(combined_features2, class_list, test_size=0.2, random_state=42)
print(f"Shape of x_train (Combined Features HOG + LBP + GLCM): {X_train2.shape}")

X_train3, X_test3, y_train3, y_test3 = train_test_split(combined_features3, class_list, test_size=0.2, random_state=42)
print(f"Shape of x_train (Combined Features LBP + Gabor + Color Hist): {X_train3.shape}")

X_train4, X_test4, y_train4, y_test4 = train_test_split(combined_features4, class_list, test_size=0.2, random_state=42)
print(f"Shape of x_train (Combined Features HOG + LBP + Color Hist): {X_train4.shape}")

X_train5, X_test5, y_train5, y_test5 = train_test_split(combined_features5, class_list, test_size=0.2, random_state=42)
print(f"Shape of X_train (Combined Features LBP + Color Hist): {X_train5.shape}")






Shape of X_train (Combined Features Gabor + HOG + Color Hist): (974, 6188)
Shape of x_train (Combined Features HOG + LBP + GLCM): (974, 5471)
Shape of x_train (Combined Features LBP + Gabor + Color Hist): (974, 839)
Shape of x_train (Combined Features HOG + LBP + Color Hist): (974, 6235)
Shape of X_train (Combined Features LBP + Color Hist): (974, 827)


In [ ]:
from sklearn.model_selection import GridSearchCV
param_grid = {
    'n_estimators': [100, 200], #, 500, 1000
    'max_depth': [None, 10, 20], #, 30
    'min_samples_split': [2, 5], #, 10
    'min_samples_leaf': [1, 2], #, 4
    'max_features': [ 'sqrt', 'log2']
}

param_grid2 = {
    'n_estimators': [500, 1000], #, 500, 1000
    'max_depth': [20, 30], #, 30
    'min_samples_split': [5,10], #, 10
    'min_samples_leaf': [2,4], #, 4
    'max_features': ['sqrt', 'log2']
}


# rf_model = GridSearchCV(estimator=RandomForestClassifier(random_state=0), param_grid=param_grid, cv=3, n_jobs=-1)
rf_model = RandomForestClassifier(n_estimators=901,min_samples_split=2,min_samples_leaf=1, random_state=0)

### Combined Training

In [ ]:
rf_model.fit(X_train, y_train)
# print(rf_model.best_params_)

# Evaluate
y_pred = rf_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")
print(classification_report(y_test, y_pred,target_names=os.listdir('/content/drive/MyDrive/dataset-coral-baru')))

Accuracy: 41.39%
                          precision    recall  f1-score   support

               Alveopora       0.00      0.00      0.00        20
                Ricordea       0.28      0.54      0.37        26
    Dendrogyra cylindrus       0.61      0.65      0.63        17
               Discosoma       0.25      0.09      0.14        32
               Euphyllia       0.32      0.38      0.35        26
              Cyphastrea       0.67      0.11      0.18        19
                Palythoa       1.00      0.17      0.29         6
Heliofungia actiniformis       0.93      0.61      0.74        23
     Caulastraea Furcata       0.60      0.60      0.60        25
       Diploria Strigosa       0.42      0.77      0.54        30
    Acropora Cervicornis       0.26      0.40      0.31        20

                accuracy                           0.41       244
               macro avg       0.49      0.39      0.38       244
            weighted avg       0.44      0.41      0.38  

In [ ]:
rf_model.fit(X_train2, y_train2)
# print(rf_model.best_params_)

y_pred2 = rf_model.predict(X_test2)
accuracy2 = accuracy_score(y_test2, y_pred2)
print(f"Accuracy for Combined Features HOG + LBP + GLCM: {accuracy2 * 100:.2f}%")
print(classification_report(y_test2, y_pred2, target_names=os.listdir('/content/drive/MyDrive/dataset-coral-baru')))

Accuracy for Combined Features HOG + LBP + GLCM: 40.16%
                          precision    recall  f1-score   support

               Alveopora       0.00      0.00      0.00        20
                Ricordea       0.24      0.58      0.34        26
    Dendrogyra cylindrus       0.60      0.53      0.56        17
               Discosoma       0.25      0.06      0.10        32
               Euphyllia       0.56      0.38      0.45        26
              Cyphastrea       0.00      0.00      0.00        19
                Palythoa       1.00      0.17      0.29         6
Heliofungia actiniformis       0.75      0.65      0.70        23
     Caulastraea Furcata       0.46      0.72      0.56        25
       Diploria Strigosa       0.39      0.60      0.47        30
    Acropora Cervicornis       0.33      0.50      0.40        20

                accuracy                           0.40       244
               macro avg       0.42      0.38      0.35       244
            weight

In [ ]:
rf_model.fit(X_train3, y_train3)
# print(rf_model.best_params_)

y_pred3 = rf_model.predict(X_test3)
accuracy3 = accuracy_score(y_test3, y_pred3)
print(f"Accuracy for Combined Features LBP + Gabor + Color Hist: {accuracy3 * 100:.2f}%")
print(classification_report(y_test3, y_pred3, target_names=os.listdir('/content/drive/MyDrive/dataset-coral-baru')))

Accuracy for Combined Features LBP + Gabor + Color Hist: 43.44%
                          precision    recall  f1-score   support

               Alveopora       0.22      0.10      0.14        20
                Ricordea       0.38      0.42      0.40        26
    Dendrogyra cylindrus       0.43      0.53      0.47        17
               Discosoma       0.42      0.16      0.23        32
               Euphyllia       0.46      0.46      0.46        26
              Cyphastrea       0.57      0.21      0.31        19
                Palythoa       0.75      0.50      0.60         6
Heliofungia actiniformis       0.58      0.48      0.52        23
     Caulastraea Furcata       0.41      0.64      0.50        25
       Diploria Strigosa       0.42      0.57      0.49        30
    Acropora Cervicornis       0.42      0.80      0.55        20

                accuracy                           0.43       244
               macro avg       0.46      0.44      0.42       244
          

In [ ]:
rf_model.fit(X_train4, y_train4)
# print(rf_model.best_params_)

y_pred4 = rf_model.predict(X_test4)
accuracy4 = accuracy_score(y_test4, y_pred4)
print(f"Accuracy for Combined Features HOG + LBP + Color Hist: {accuracy4 * 100:.2f}%")
print(classification_report(y_test4, y_pred4, target_names=os.listdir('/content/drive/MyDrive/dataset-coral-baru')))
print(classification_report(y_test2, y_pred2, target_names=os.listdir('/content/drive/MyDrive/Computer Vision/dataset-coral-baru')))

Accuracy for Combined Features HOG + LBP + Color Hist: 44.67%
                          precision    recall  f1-score   support

               Alveopora       0.67      0.10      0.17        20
                Ricordea       0.33      0.65      0.44        26
    Dendrogyra cylindrus       0.59      0.59      0.59        17
               Discosoma       0.36      0.12      0.19        32
               Euphyllia       0.34      0.38      0.36        26
              Cyphastrea       1.00      0.05      0.10        19
                Palythoa       1.00      0.17      0.29         6
Heliofungia actiniformis       0.87      0.57      0.68        23
     Caulastraea Furcata       0.52      0.60      0.56        25
       Diploria Strigosa       0.43      0.83      0.57        30
    Acropora Cervicornis       0.38      0.55      0.45        20

                accuracy                           0.45       244
               macro avg       0.59      0.42      0.40       244
            

In [ ]:
rf_model.fit(X_train5, y_train5)
# print(rf_model.best_params_)

y_pred5 = rf_model.predict(X_test5)
accuracy5 = accuracy_score(y_test5, y_pred5)
print(f"Accuracy for Combined Features LBP+COLOR_HIST: {accuracy5 * 100:.2f}%")
print(classification_report(y_test6, y_pred6, target_names=os.listdir('/content/drive/MyDrive/dataset-coral-baru')))


Accuracy for Combined Features LBP+COLOR_HIST: 38.11%
                          precision    recall  f1-score   support

               Alveopora       0.00      0.00      0.00        20
                Ricordea       0.23      0.58      0.33        26
    Dendrogyra cylindrus       0.91      0.59      0.71        17
               Discosoma       0.20      0.06      0.10        32
               Euphyllia       0.35      0.42      0.39        26
              Cyphastrea       0.50      0.05      0.10        19
                Palythoa       1.00      0.17      0.29         6
Heliofungia actiniformis       0.67      0.52      0.59        23
     Caulastraea Furcata       0.43      0.40      0.42        25
       Diploria Strigosa       0.43      0.77      0.55        30
    Acropora Cervicornis       0.31      0.45      0.37        20

                accuracy                           0.39       244
               macro avg       0.46      0.36      0.35       244
            weighted

In [ ]:
print("DENGAN CLAHE PARAM GRID2")
print(f"Accuracy Gabor + HOG + Color Hist: {accuracy * 100:.2f}%")
print(f"Accuracy HOG + LBP + GLCM : {accuracy2 * 100:.2f}%")
print(f"Accuracy LBP + Gabor + Color Hist: {accuracy3 * 100:.2f}%")
print(f"Accuracy HOG + LBP + Color Hist: {accuracy4 * 100:.2f}%")
print(f"Accuracy LBP + Color Hist: {accuracy5 * 100:.2f}%")




DENGAN CLAHE PARAM GRID2
Accuracy Gabor + HOG + Color Hist: 41.39%
Accuracy HOG + LBP + GLCM : 40.16%
Accuracy LBP + Gabor + Color Hist: 43.44%
Accuracy HOG + LBP + Color Hist: 44.67%
Accuracy LBP + Color Hist: 38.11%


### Single Training

In [ ]:
rf_model.fit(x_train_hog, y_train_hog)
y_pred_hog = rf_model.predict(x_test_hog)
accuracy_hog = accuracy_score(y_test_hog, y_pred_hog)
print(f"Accuracy for HOG: {accuracy_hog * 100:.2f}%")
print(classification_report(y_test_hog, y_pred_hog, target_names=os.listdir('/content/drive/MyDrive/dataset-coral-baru')))

Accuracy for HOG: 34.84%
                          precision    recall  f1-score   support

               Alveopora       0.25      0.05      0.08        20
                Ricordea       0.24      0.58      0.34        26
    Dendrogyra cylindrus       1.00      0.24      0.38        17
               Discosoma       0.30      0.09      0.14        32
               Euphyllia       0.47      0.31      0.37        26
              Cyphastrea       0.00      0.00      0.00        19
                Palythoa       1.00      0.17      0.29         6
Heliofungia actiniformis       0.71      0.52      0.60        23
     Caulastraea Furcata       0.54      0.56      0.55        25
       Diploria Strigosa       0.30      0.60      0.40        30
    Acropora Cervicornis       0.23      0.45      0.30        20

                accuracy                           0.35       244
               macro avg       0.46      0.32      0.31       244
            weighted avg       0.41      0.35    

In [ ]:
rf_model.fit(x_train_lbp, y_train_lbp)
y_pred_lbp = rf_model.predict(x_test_lbp)
accuracy_lbp = accuracy_score(y_test_lbp, y_pred_lbp)
print(f"Accuracy for LBP: {accuracy_lbp * 100:.2f}%")
print(classification_report(y_test_lbp, y_pred_lbp, target_names=os.listdir('/content/drive/MyDrive/dataset-coral-baru')))


Accuracy for LBP: 30.74%
                          precision    recall  f1-score   support

               Alveopora       0.15      0.10      0.12        20
                Ricordea       0.38      0.23      0.29        26
    Dendrogyra cylindrus       0.41      0.65      0.50        17
               Discosoma       0.06      0.03      0.04        32
               Euphyllia       0.30      0.35      0.32        26
              Cyphastrea       0.31      0.26      0.29        19
                Palythoa       0.29      0.33      0.31         6
Heliofungia actiniformis       0.35      0.30      0.33        23
     Caulastraea Furcata       0.33      0.36      0.35        25
       Diploria Strigosa       0.30      0.43      0.35        30
    Acropora Cervicornis       0.38      0.50      0.43        20

                accuracy                           0.31       244
               macro avg       0.30      0.32      0.30       244
            weighted avg       0.29      0.31    

In [ ]:
rf_model.fit(x_train_glcm, y_train_glcm)
y_pred_glcm = rf_model.predict(x_test_glcm)
accuracy_glcm = accuracy_score(y_test_glcm, y_pred_glcm)
print(f"Accuracy for GLCM: {accuracy_glcm * 100:.2f}%")
print(classification_report(y_test_glcm, y_pred_glcm, target_names=os.listdir('/content/drive/MyDrive/dataset-coral-baru')))

Accuracy for GLCM: 27.05%
                          precision    recall  f1-score   support

               Alveopora       0.11      0.15      0.13        20
                Ricordea       0.21      0.23      0.22        26
    Dendrogyra cylindrus       0.40      0.35      0.38        17
               Discosoma       0.00      0.00      0.00        32
               Euphyllia       0.29      0.27      0.28        26
              Cyphastrea       0.27      0.16      0.20        19
                Palythoa       0.17      0.17      0.17         6
Heliofungia actiniformis       0.44      0.30      0.36        23
     Caulastraea Furcata       0.35      0.48      0.41        25
       Diploria Strigosa       0.28      0.40      0.33        30
    Acropora Cervicornis       0.32      0.45      0.38        20

                accuracy                           0.27       244
               macro avg       0.26      0.27      0.26       244
            weighted avg       0.25      0.27   

In [ ]:
rf_model.fit(x_train_gabor, y_train_gabor)
y_pred_gabor = rf_model.predict(x_test_gabor)
accuracy_gabor = accuracy_score(y_test_gabor, y_pred_gabor)
print(f"Accuracy for Gabor: {accuracy_gabor * 100:.2f}%")
print(classification_report(y_test_gabor, y_pred_gabor, target_names=os.listdir('/content/drive/MyDrive/dataset-coral-baru')))

Accuracy for Gabor: 30.33%
                          precision    recall  f1-score   support

               Alveopora       0.16      0.15      0.15        20
                Ricordea       0.23      0.27      0.25        26
    Dendrogyra cylindrus       0.47      0.53      0.50        17
               Discosoma       0.17      0.12      0.14        32
               Euphyllia       0.20      0.19      0.20        26
              Cyphastrea       0.18      0.11      0.13        19
                Palythoa       0.25      0.17      0.20         6
Heliofungia actiniformis       0.41      0.30      0.35        23
     Caulastraea Furcata       0.36      0.32      0.34        25
       Diploria Strigosa       0.51      0.67      0.58        30
    Acropora Cervicornis       0.24      0.40      0.30        20

                accuracy                           0.30       244
               macro avg       0.29      0.29      0.29       244
            weighted avg       0.29      0.30  

In [ ]:
rf_model.fit(x_train_color_hist, y_train_color_hist)
y_pred_color_hist = rf_model.predict(x_test_color_hist)
accuracy_color_hist = accuracy_score(y_test_color_hist, y_pred_color_hist)
print(f"Accuracy for Gabor: {accuracy_color_hist * 100:.2f}%")
print(classification_report(y_test_color_hist, y_pred_color_hist, target_names=os.listdir('/content/drive/MyDrive/dataset-coral-baru')))

Accuracy for Gabor: 34.43%
                          precision    recall  f1-score   support

               Alveopora       0.57      0.20      0.30        20
                Ricordea       0.22      0.23      0.23        26
    Dendrogyra cylindrus       0.40      0.47      0.43        17
               Discosoma       0.24      0.12      0.16        32
               Euphyllia       0.38      0.38      0.38        26
              Cyphastrea       0.40      0.11      0.17        19
                Palythoa       0.33      0.17      0.22         6
Heliofungia actiniformis       0.34      0.43      0.38        23
     Caulastraea Furcata       0.32      0.44      0.37        25
       Diploria Strigosa       0.40      0.57      0.47        30
    Acropora Cervicornis       0.33      0.55      0.42        20

                accuracy                           0.34       244
               macro avg       0.36      0.33      0.32       244
            weighted avg       0.35      0.34  

In [ ]:
print("DENGAN CLAHE")
print(f"Accuracy HOG: {accuracy_hog * 100:.2f}%")
print(f"Accuracy LBP: {accuracy_lbp * 100:.2f}%")
print(f"Accuracy GLCM: {accuracy_glcm * 100:.2f}%")
print(f"Accuracy Gabor: {accuracy_gabor * 100:.2f}%")
print(f"Accuracy Color_hist: {accuracy_color_hist * 100:.2f}%")

DENGAN CLAHE
Accuracy HOG: 34.84%
Accuracy LBP: 30.74%
Accuracy GLCM: 27.05%
Accuracy Gabor: 30.33%
Accuracy Color_hist: 34.43%
